# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [59]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,-0.27,98,100,6.20,GS,1694886603
1,1,ca mau,9.1769,105.1500,24.94,90,55,1.68,VN,1694886603
2,2,invercargill,-46.4000,168.3500,9.45,83,100,3.13,NZ,1694886504
3,3,ilulissat,69.2167,-51.1000,4.01,70,75,2.57,GL,1694886604
4,4,dudinka,69.4058,86.1778,2.57,77,100,3.90,RU,1694886604


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [104]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 900,
    size = "Humidity",
    scale = .65,
    color = "City")
# Display the map
    
    
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [107]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_ideal = city_data_df[(city_data_df['Humidity'] <80)& (city_data_df['Cloudiness'] <5) & (city_data_df['Max Temp'] > 25)& (city_data_df['Country'] == 'US')]
                                                                                                    

# Drop any rows with null values
city_data_df_ideal.dropna(inplace = True)

# Display sample data
city_data_df_ideal

/var/folders/nz/y789jt4x0ns1x41s7zsq86cc0000gn/T/ipykernel_23762/1525904764.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_data_df_ideal.dropna(inplace = True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,33,fort bragg,35.1390,-79.0060,28.28,39,0,2.06,US,1694886611
60,60,hamilton,39.1834,-84.5333,26.23,47,0,2.06,US,1694886450
79,79,laguna,38.4210,-121.4238,25.17,67,0,5.14,US,1694886620
341,341,greenville,35.6127,-77.3663,26.02,46,0,0.00,US,1694886673
381,381,hermiston,45.8404,-119.2895,29.58,42,0,1.54,US,1694886578
403,403,oasis,33.4659,-116.0989,37.69,51,0,4.63,US,1694886687
417,417,morehead city,34.7229,-76.7260,27.80,52,0,2.24,US,1694886690
435,435,south venice,27.0531,-82.4243,33.92,79,0,4.47,US,1694886693
449,449,tanque verde,32.2517,-110.7373,34.39,18,0,2.24,US,1694886697
514,514,poolesville,39.1459,-77.4169,27.07,45,0,2.06,US,1694886625


### Step 3: Create a new DataFrame called `hotel_df`.

In [108]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_ideal.copy()
hotel_df = hotel_df[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
33,fort bragg,US,35.1390,-79.0060,39,
60,hamilton,US,39.1834,-84.5333,47,
79,laguna,US,38.4210,-121.4238,67,
341,greenville,US,35.6127,-77.3663,46,
381,hermiston,US,45.8404,-119.2895,42,
403,oasis,US,33.4659,-116.0989,51,
417,morehead city,US,34.7229,-76.7260,52,
435,south venice,US,27.0531,-82.4243,79,
449,tanque verde,US,32.2517,-110.7373,18,
514,poolesville,US,39.1459,-77.4169,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [109]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "filter":'',
    "bias":'',
    "categories":'accommodation.hotel',
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort bragg - nearest hotel: Airborne Inn Lodging
hamilton - nearest hotel: North Vista Manor
laguna - nearest hotel: Holiday Inn Express & Suites
greenville - nearest hotel: Quality Inn
hermiston - nearest hotel: Holiday Inn Express & Suites;Holiday Inn Express & Suites Hermiston Downtown
oasis - nearest hotel: No hotel found
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
south venice - nearest hotel: Kon-Tiki Motel
tanque verde - nearest hotel: No hotel found
poolesville - nearest hotel: Lansdowne Resort and Spa


,City,Country,Lat,Lng,Humidity,Hotel Name
33,fort bragg,US,35.1390,-79.0060,39,Airborne Inn Lodging
60,hamilton,US,39.1834,-84.5333,47,North Vista Manor
79,laguna,US,38.4210,-121.4238,67,Holiday Inn Express & Suites
341,greenville,US,35.6127,-77.3663,46,Quality Inn
381,hermiston,US,45.8404,-119.2895,42,Holiday Inn Express & Suites;Holiday Inn Expre...
403,oasis,US,33.4659,-116.0989,51,No hotel found
417,morehead city,US,34.7229,-76.7260,52,"Bask Hotel Big Rock Landing, Trademark Collect..."
435,south venice,US,27.0531,-82.4243,79,Kon-Tiki Motel
449,tanque verde,US,32.2517,-110.7373,18,No hotel found
514,poolesville,US,39.1459,-77.4169,45,Lansdowne Resort and Spa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [121]:
%%capture --no-display

# Configure the map plot
map_plot_9 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 1000,
    scale = 10,
    hover_cols = ["City",'Country','Hotel Name']
)

# Display the map
map_plot_9

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)